In [31]:
from torchengine import AnalyticalSet, Function, EliminateAnalysis, EliminateAnalysisMergeResiduals, ElimResidual, generate_eval_and_gradient
from sympy.utilities.lambdify import implemented_function
import torch
import numpy as np
import sympy as sp
# Set the print options
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})

In [127]:
       
class AnalyticalSetRaw():
    def __init__(self, analysis, residuals, outputs) -> None:
        self.analysis = analysis
        self.residual = residuals
        self.outputs = outputs

def initialize_tensor(dict, indices):
    output = torch.tensor([0.]*len(torch.cat(tuple(indices.values()))))
    for key,val in dict.items():
        output[indices[key]] = val
    return output

In [156]:
indices1ex = {
    'x': torch.tensor([0, 1, 2]),
    'a1': torch.tensor([3, 4]),
    'a2': torch.tensor([5])
}
indices2ex = {
    'x': torch.tensor([0, 3, 4]),
    'u1': torch.tensor([1, 2]),
    'u2': torch.tensor([5, 6]),
    'a1': torch.tensor([7, 8]),
}
indices2ex_len = len(torch.cat(tuple(indices2ex.values())))
copy_over_indices = ['x']
copy_indices_tuple = [(indices1ex[key], indices2ex[key]) for key in copy_over_indices]
def transfer_value(xfrom, xto, copy_indices_tuple):
    for indices_in,indices_out  in copy_indices_tuple:
        xto[indices_out] = xfrom[indices_in]
    return xto

In [161]:
# xin = initialize_tensor({'x': torch.tensor([1.,2,3]), 'a1': torch.tensor([4.,5])}, indices1ex)
# xzero = torch.empty(indices2ex_len)
# xout = transfer_value(xin, xzero, copy_indices_tuple)
# set1 = AnalyticalSet((('x',), ('a1',), lambda x: (x[:2],)), indices2ex)
# results = set1.analysis(xout)

tensor([1., 2., 3., 1., 2., 0.])

In [162]:
mu = 3.986e14
Rearth = 6378
analysis1 = (('h',), ('a',), lambda h: (h+Rearth)*1e3)
analysis2 = (('a',), ('T',), lambda a: 2*np.pi*(a**3/mu)**0.5)
analysis3 = (('a',), ('g',), lambda a: 1/np.pi*np.arccos(Rearth*1e3/a))
analysis4 = (('g',), ('d',), lambda g: g+0.5)
analysis5 = (('h',), ('r',), lambda h: (h**2+2*Rearth*h)**0.5)

varorder = ['h', 'x', 'R', 'a', 'y', 'T', 'g', 'd', 'r']
indices = {elt: torch.tensor([i]) for i, elt in enumerate(varorder)}

In [188]:
x0 = initialize_tensor({'h': 200, 'a': 1.1*Rearth*1e3, 'g':0.1}, indices)
set1 = AnalyticalSet(analysis1, indices)
set2 = AnalyticalSet(analysis2, indices)
set3 = AnalyticalSet(analysis3, indices)
set4 = AnalyticalSet(analysis4, indices)
set5 = AnalyticalSet(analysis5, indices)

orbit_analysis = EliminateAnalysis([set1.analysis, set2.analysis, set3.analysis, set4.analysis, set5.analysis],[])
orbit_residuals = EliminateAnalysis([], [set1.residual, set2.residual, set3.residual, set4.residual, set5.residual])
all_outputs = [set1.outputs, set2.outputs, set3.outputs, set4.outputs, set5.outputs]
Orbit = AnalyticalSetRaw(orbit_analysis, orbit_residuals, all_outputs) # Maybe should be indices too?
output = Orbit.analysis(x0)

output.numpy()

array([200, 0, 0, 6578000, 0, 5309.48, 0.08, 0.58, 1609.72], dtype=float32)

In [177]:
obj = Function((('r',), lambda r: r), indices)

In [198]:
# IDF for one shared variable for Orbit, say for g
inputs = varorder
input_indices = torch.cat([indices[i] for i in inputs]) #flatten

def IDFres(analysis, functions, output_indices):
    def forward(x):
        y = x.clone()
        r = []
        for a in analysis:
            y[output_indices] = a(x)[output_indices]
            r.append(y[output_indices]-x[output_indices])
        return y, torch.cat(r), tuple(f(y) for f in functions)
    return forward

outputs = ('g',)
output_indices = torch.cat([indices[i] for i in outputs])

idfdata = IDFres([Orbit.analysis], [obj], output_indices)

def transfer_vars(indicesin, indicesout):
    copy_over_indices = indicesin.keys()
    copy_indices_tuple = [(indicesin[key], indicesout[key]) for key in copy_over_indices]
    indicesout_len = len(torch.cat(tuple(indicesout.values())))
    def forward(x):
        xzero = torch.empty(indicesout_len)
        output = transfer_value(x, xzero, copy_indices_tuple)
        return output
    return forward

system_ins = ['h']
indices_in = {elt: indices[elt] for elt in system_ins}
mdf_eliminatedvars = transfer_vars(indices_in, indices)
MDF = EliminateAnalysis([mdf_eliminatedvars, Orbit.analysis], [obj])


In [200]:
MDF(torch.tensor([200.]))

tensor([1609.7205])

In [197]:
y, eqval, objval = idfdata(x0)
x0.numpy(), y.numpy()

(array([200, 0, 0, 7015800, 0, 0, 0.1, 0, 0], dtype=float32),
 array([200, 0, 0, 7015800, 0, 0, 0.08, 0, 0], dtype=float32))

# Solving a system of equations

In [2]:
indices = {
    'mt': torch.tensor([0]),
    'ms': torch.tensor([1]),
    'mp': torch.tensor([2])
}

analysis6 = (('mt',), ('ms',), lambda mt: 0.2*mt)
analysis7 = (('ms','mp'), ('mt',), lambda ms, mp: 0.2*ms+mp)
ineqcon1 = (('mp',), lambda mp: -mp/0.5+1)
objfunc = (('mt',), lambda mt: mt)

set6 = AnalyticalSet(analysis6, indices)
set7 = AnalyticalSet(analysis7, indices)
con1 = Function(ineqcon1, indices)
obj = Function(objfunc, indices)

coupled = EliminateAnalysis([],[set6.residual, set7.residual])
solver = ElimResidual(coupled, torch.tensor([0,1]), [2])

AttributeError: 'list' object has no attribute 'items'

In [248]:
obj_mp = EliminateAnalysis([solver], [obj])
x = torch.tensor([1.,2.,0.7])
obj_mp(x)

tensor([0.7292])

In [249]:
x

tensor([0.7292, 0.1458, 0.7000])

### Sellar

In [1]:
from torchengine import AnalyticalSet, Function, EliminateAnalysis, EliminateAnalysisMergeResiduals, ElimResidual
from scipy.optimize import minimize
import numpy as np
import torch
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})

In [87]:
sellarobj = (('x2', 'x3', 'u1', 'u2'), lambda x2, x3, u1, u2: x3**2 + x2 + u1 + np.exp(-u2))
sellar1 = (('x1', 'x2', 'x3', 'u2'), ('u1',), lambda x1, x2, x3, u2: x1**2 + x2 + x3 - 0.2*u2)
sellar2 = (('x1', 'x2', 'u1'), ('u2',), lambda x1, x2, u1: u1**0.5 + x1 + x2)
ineqcon1 = (('u1',), lambda u1: 1-u1/3.16)
ineqcon2 = (('u2',), lambda u2: u2/24-1)

varorder = ['x1','x2','x3','u1','u2']
indices = {elt: torch.tensor([i]) for i, elt in enumerate(varorder)}

# Disciplines
# pipelines
# [{1},{2},{3,4,5},{6,7}], obj, eq, ineq
# -> variant one: [1,2], [3,4,5,6,7] # 2 options: reduced AAO / reduced IDF
# ## reduced AAO
# neweq = EliminateAnalysisMergeResiduals([], [3.r,4.r,5.r,6.r,7.r,eq]) 
# ## reduced IDF
# idf_eq = Parallel([3.a,4.a,5.a,6.a,7.a])
# neweq = EliminateAnalysisMergeResiduals([], [idf_eq, eq]) 
# ##BOTH:
# objf, eqf, ineqf = EliminateAnalysisMergeResiduals([1.a,2.a], [obj, neweq, ineq])
# -> variant two: [1,2, ([3,4],5), (6,7)] # reduced MDF (tearing based)
# block1 = EliminateAnalysisMergeResiduals([3.a,4.a], [5.r])
# block2 = EliminateAnalysisMergeResiduals([], [6.r, 7.r])
# objf, eqf, ineqf = EliminateAnalysisMergeResiduals([1,2, block1, block2], [ obj, eq, ineq])
# -> variant three: (1,2,3,4,5,6,7) # baseline MDF
# objf, eqf, ineqf = EliminateAnalysisMergeResiduals([1,2, 3, 4, 5, 6, 7], [ obj, eq, ineq])

set1 = AnalyticalSet(sellar1, indices)
set2 = AnalyticalSet(sellar2, indices)

coupled = EliminateAnalysisMergeResiduals([],[set1.residual, set2.residual])
coupledvars = ['u1','u2']
solver = ElimResidual(coupled, coupledvars, indices)

# Optimization problem
con1 = Function(ineqcon1, indices)
con2 = Function(ineqcon2, indices)
ineqcons = EliminateAnalysisMergeResiduals([], [con1, con2])
obj = Function(sellarobj, indices)

### MDA

In [11]:
x0 = torch.tensor([2.,0.,0.,0.1,0.1], dtype=torch.float64) # Important for finite differences
sol = solver(x0)
sol.numpy(), coupled(sol).numpy()

(array([2, 0, 0, 3.24, 3.8]), array([-0, 0]))

## Build optimization problem

In [56]:
def generate_optim_functions(optim_funcs, solvefor_indices, x):
    def eval_all(y):
        x[solvefor_indices] = torch.from_numpy(y).to(x.dtype)
        objval, ineqval, eqval = optim_funcs(x)
        return objval.item(), -ineqval, -eqval
    
    def obj_function(y):
        objval, _, _ = eval_all(y)
        return objval

    def ineq_function(y=None):
        _, ineqval, _ = eval_all(y)
        return ineqval
    
    def eq_function(y=None):
        _, _, eqval = eval_all(y)
        return eqval

    xguess = x[solvefor_indices].numpy()
    
    return xguess, obj_function, ineq_function, eq_function

## MDF

In [57]:
bnds = [(0, 10), (0, 10), (0, 10), (3.16, None), (None, 24)]
optim_vars = ['x1','x2','x3']
optim_indices = torch.cat([indices[elt] for elt in optim_vars])
bnds_problem = [bnds[elt] for elt in optim_indices]

optim_funcs_MDF = EliminateAnalysis([solver], [obj, ineqcons, lambda x: torch.tensor([])])
x0 = torch.tensor([0.5,.5,0.5,0.1,0.1], dtype=torch.float64) # needed for solver
xguess, obj_function, ineq_function, eq_function = generate_optim_functions(optim_funcs_MDF, optim_indices, x0)
constraints = [{'type': 'ineq', 'fun': ineq_function}]

In [58]:
# Solve the optimization problem
minimize(obj_function, xguess, bounds=bnds_problem, constraints=constraints, method='SLSQP')

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 3.183393952217446
       x: [ 1.978e+00  2.316e-15  2.031e-15]
     nit: 6
     jac: [ 3.508e+00  1.729e+00  9.405e-01]
    nfev: 25
    njev: 6

In [59]:
obj(x0).item(), ineqcons(x0).numpy(), x0.numpy()

(3.1833939662321673, array([-0, -0.84]), array([1.98, 0, 0, 3.16, 3.76]))

# IDF

In [60]:
def IDFres(sets, functions, indices):
    outputidx_and_analyses = [(torch.cat([indices[i] for i in s.outputs]),s.analysis) for s in sets]
    def forward(x):
        y = x.clone()
        r = []
        for output_indices, a in outputidx_and_analyses:
            y[output_indices] = a(x)[output_indices]
            r.append(y[output_indices]-x[output_indices])
        obj, ineq, eq = [f(y) for f in functions]
        return obj, ineq, torch.cat((eq, torch.cat(r)))
    return forward

In [65]:
optim_vars = ['x1','x2','x3', 'u1', 'u2']
optim_indices = torch.cat([indices[elt] for elt in optim_vars])
bnds_problem = [bnds[elt] for elt in optim_indices]

optim_funcs_IDF = IDFres([set1, set2], [obj, ineqcons, lambda x: torch.tensor([])], indices)
x0 = torch.tensor([0.5,.5,0.5,0.1,0.1], dtype=torch.float64) # needed for solver
xguess, obj_function, ineq_function, eq_function = generate_optim_functions(optim_funcs_IDF, optim_indices, x0)
constraints = [{'type': 'ineq', 'fun': ineq_function}, {'type': 'eq', 'fun': eq_function}]

In [66]:
eq_function(xguess)

tensor([-1.1300, -1.2162], dtype=torch.float64)

In [67]:
# Solve the optimization problem
minimize(obj_function, xguess, bounds=bnds_problem, constraints=constraints, method='SLSQP')

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 3.183394101954299
       x: [ 1.978e+00  0.000e+00  2.087e-15  3.160e+00  3.755e+00]
     nit: 7
     jac: [ 3.932e+00  1.977e+00  1.000e+00 -6.580e-03 -2.000e-01]
    nfev: 42
    njev: 7

In [68]:
obj(x0).item(), ineqcons(x0).numpy(), x0.numpy()

(3.18339395034975, array([0, -0.84]), array([1.98, 0, 0, 3.16, 3.76]))

## AAO

In [81]:
optim_vars = ['x1','x2','x3', 'u1', 'u2']
optim_indices = torch.cat([indices[elt] for elt in optim_vars])
bnds_problem = [bnds[elt] for elt in optim_indices]

optim_funcs_IDF = IDFres([set1, set2], [obj, ineqcons, lambda x: torch.tensor([])], indices)
x0 = torch.tensor([0.5,.5,0.5,0.1,0.1], dtype=torch.float64) # needed for solver
optim_funcs_AAO = lambda x: [obj(x), ineqcons(x), coupled(x)]
xguess, obj_function, ineq_function, eq_function = generate_optim_functions(optim_funcs_AAO, optim_indices, x0)
constraints = [{'type': 'ineq', 'fun': ineq_function}, {'type': 'eq', 'fun': eq_function}]

In [85]:
# Solve the optimization problem
minimize(obj_function, xguess, bounds=bnds_problem, constraints=constraints, method='SLSQP')

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 3.1833939516406087
       x: [ 1.978e+00  4.925e-16  3.788e-16  3.160e+00  3.755e+00]
     nit: 6
     jac: [ 0.000e+00  1.000e+00  2.980e-08  1.000e+00 -2.339e-02]
    nfev: 37
    njev: 6

In [86]:
obj(x0).item(), ineqcons(x0).numpy(), x0.numpy()

(3.1833939516406087, array([-0, -0.84]), array([1.98, 0, 0, 3.16, 3.76]))

In [61]:
# delta = 1e-7
# (obj_function(xguess+np.eye(3)[2]*delta)-obj_function(xguess))/delta

### Desired modeling syntax

In [ ]:
Orbit = AnalysisIntersection() # we cannot know what the shared variables will be until all have sets have been added. This is why .analysis and .residual need to be calculated at runtime
h = Var('h', 400e3) 
a = Orbit.Var('a', h + R, setid=1)
T = Orbit.Var('T', 2*np.pi*(a**3/μ)**0.5, setid=2)
g = Orbit.Var('g', 1/np.pi*sp.acos(R/a), setid=3)
d = Orbit.Var('d', g+0.5, setid=4)
r = Orbit.Var('r', (h**2+2*R*h)**0.5, setid=5)

Power = AnalysisIntersection()
eta_A = Par(r'\eta_A', 0.3)
rho_A = Par(r'\rho_A', 10) 
rho_b = Par(r'\rho_b', 0.002)
P_l = Par('P_l', 12, 'W')
A = Var('A', 0.05)
m_A = Power.Var('m_A', rho_A*A, setid=6)
P_c = Power.Var('P_c', d*A*Q*eta_A, setid=7)
P_T = Power.Var('P_T', P_c-P_l, setid=8) 
E_b = Power.Var('E_b', P_c*T/d, setid=9)
m_b = Power.Var('m_b', rho_b*E_b, setid=10)

Payload = AnalysisIntersection()
X_r = Var('X_r', 5)
rho_p = Par(r'\rho_p', 2) 
l_v = Par('l_v', 500, 'nm')
B = Par('B', 8)
N = Par('N', 2000)
D_p = Payload.Var('D_p', 1.22*l_v*h/X_r, setid=11)
D = Payload.Var('D', 2*np.pi*R*B*N/X_r, setid=12)
m_p = Payload.Var('m_p', rho_p*D_p**1.5, setid=13)

# Optimization relevant functions. These are needed for IDF/MDF
ineq = [Xr - 5]
obj = m_p + m_b

# IDF would find shared variables accross all analyses and then generate
# the equality constraints. The resulting form (when using information form one level below) would look like:
([1,2,4,7,9,11,12], 3, 5, 6, 8, 10, 13)